In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install soynlp -q

In [ ]:
from soynlp.word import WordExtractor
from soynlp import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
import urllib.request
import re
import csv
import pickle
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/02)2024-1학기/2_자연어처리/00_개인 프로젝트 논문 (웹툰굿즈)/데이터 및 코드/★라벨링_통합본.xlsx')
df = df.sort_values(by='brand')
df = df[df['brand']=='마루는강쥐']
df

,brand,product_name,review,rating
8998,마루는강쥐,롱스티커,귀여워요!!!!!!!!!!!!!!!,1
7465,마루는강쥐,빅데코 스티커,너무귀엽고 또 사러올거에요😊,1
7464,마루는강쥐,빅데코 스티커,진짜 넘 귀여워요 마루랑 함께 일하는 기분 짱!,1
7463,마루는강쥐,빅데코 스티커,흑흑 귀여운 마루가 커지다니.. 왕크왕귀💓💓 \n진심 캐릭터 굿즈에 30000원을 ...,1
7462,마루는강쥐,빅데코 스티커,빅데코 스티커 잘 보여서 왕 크니까 왕 귀여워요,1
...,...,...,...,...
6481,마루는강쥐,인덱스노트,하 .. 너무 귀여워요 마루 사랑해,1
6480,마루는강쥐,인덱스노트,너무너무 귀여워서 재구매 의사 있습니다 ㅠ,1
6483,마루는강쥐,인덱스노트,너~~~~~~~~~~무 귀여워요!!!!!!!!!!! 후회없는 구매입니다ㅎㅎ,1
6479,마루는강쥐,인덱스노트,일단 개..귀여워요,1


In [ ]:
df1 = df[df['rating']==1]
df1_그립톡 = df1[df1['product_name']=='그립톡 (2종)'] #그립톡 (2종) 긍정 리뷰
df1_롱스티커 = df1[df1['product_name']=='롱스티커'] #롱스티커 긍정 리뷰
df1_인형키링 = df1[df1['product_name']=='미니 인형키링'] #미니 인형키링 긍정 리뷰
df1_스프링노트 = df1[df1['product_name']=='스프링노트'] #스프링노트 긍정 리뷰
df1_사각필통 = df1[df1['product_name']=='사각필통'] #사각필통 긍정 리뷰

In [ ]:
df0 = df[df['rating']==0]
df0_그립톡 = df0[df0['product_name']=='그립톡 (2종)'] #그립톡 (2종) 부정 리뷰
df0_롱스티커 = df0[df0['product_name']=='롱스티커'] #롱스티커 부정 리뷰
df0_인형키링 = df0[df0['product_name']=='미니 인형키링'] #미니 인형키링 부정 리뷰
df0_스프링노트 = df0[df0['product_name']=='스프링노트'] #스프링노트 부정 리뷰
df0_사각필통 = df0[df0['product_name']=='사각필통'] #사각필통 부정 리뷰

그립톡 (2종)

In [ ]:
#긍정 전처리
df1_그립톡["cut_special_character"]=df1_그립톡['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df1_그립톡["cut_korean_consonant"]=df1_그립톡["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df1_그립톡["fiterd_data"]=df1_그립톡["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

#부정 전처리
df0_그립톡["cut_special_character"]=df0_그립톡['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df0_그립톡["cut_korean_consonant"]=df0_그립톡["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df0_그립톡["fiterd_data"]=df0_그립톡["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

<ipython-input-5-0e6ce2674b87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_그립톡["cut_special_character"]=df1_그립톡['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
<ipython-input-5-0e6ce2674b87>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_그립톡["cut_korean_consonant"]=df1_그립톡["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
<ipython-input-5-0e6ce2674b87>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
review1 = listToString(df1_그립톡['fiterd_data'])
review0 = listToString(df0_그립톡['fiterd_data'])

In [ ]:
!pip install keybert -q

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

doc1 = review1 #긍정
doc0 = review0 #부정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(len(doc1), len(doc0))

68444 18989


In [ ]:
#MMR(Maximal Margimal Relevance) 긍정
keywords_mmr_1 = kw_model.extract_keywords(doc1,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_1

[('염둥아마루에요 만좁의만족의믄쥭 인간마루다 완전', 0.6543),
 ('염둥아마루에요 만좁의만족의믄쥭', 0.6264),
 ('참을수없는 염둥아마루에요 만좁의만족의믄쥭', 0.6241),
 ('참을수없는 염둥아마루에요 만좁의만족의믄쥭 인간마루다', 0.6161),
 ('염둥아마루에요 만좁의만족의믄쥭 인간마루다', 0.6141),
 ('마루 참을수없는 염둥아마루에요 만좁의만족의믄쥭', 0.6133),
 ('만좁의만족의믄쥭 인간마루다 완전', 0.6121),
 ('한도초과에용 귀여워요 정품그립톡이라 그립감도', 0.6101),
 ('사랑해요 인간마루한개더살걸 힝구힝구 너무', 0.6001),
 ('판매해주셔서 무사히 배송해주셔서 감사합니다', 0.5998),
 ('넘넘기여워요 두개나삿어요 친구 생일선물용이랑', 0.5896),
 ('문의하신 분들처럼 처음에 1단에서', 0.5878),
 ('마루상은 긔염슴미다 완전', 0.5798),
 ('너무 만족스럽습니다 화면이랑', 0.5735),
 ('편해요 이번에는 맥세이프로 만들어서', 0.5706),
 ('내주세용 핸드폰에 써서 사무실', 0.5673),
 ('킹랑스러움을 대박임 마루 너무너무너무', 0.5664),
 ('행복하시고 오래사시고 가정과', 0.5636),
 ('행복해 친구가 마루팬이라 생일', 0.5632),
 ('만족했습니다 진짜 맹숭한 표정의', 0.5394)]

In [ ]:
#MMR(Maximal Margimal Relevance) 부정
keywords_mmr_0 = kw_model.extract_keywords(doc0,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_0

[('만들어주세용 마루는 정말 너무', 0.6251),
 ('사줬는데 완전 만족해요 금액대가', 0.6246),
 ('너무너무 귀여워요 마음에들어요 생각보다', 0.6242),
 ('들어가요 너무너무 만족합니다 친구가', 0.6198),
 ('만족해하는 그립톡 그러나 맥세이프와', 0.6187),
 ('모양도 만들어주세여 마루 너무너무', 0.6181),
 ('좋아서 만족입니다 인간마루 무표정', 0.6125),
 ('너무너무너무 귀여워요 아이폰 맥세이프용도', 0.61),
 ('많이 만들어주시고 이모티콘도 만들어주시면', 0.6091),
 ('상품은 만족하지만 두번다시는', 0.6083),
 ('마루인걸 일단 생각을 하는지', 0.6058),
 ('작가님 무병장수하세요 마루가 저의', 0.6036),
 ('마음에들어요 생각보다 커요 마루는', 0.6026),
 ('물건 보낼 확인하시고 보내', 0.6005),
 ('손에 마루가 가격은 사악하지만', 0.5981),
 ('실물과 제일 가까운 색감입니다', 0.5972),
 ('택배측에서 알아서 수거한다는 것만', 0.5915),
 ('추천함돠 귀엽지만 접착력이 별로입니다', 0.5899),
 ('디자인은 정말 맘에 듭니다', 0.5887),
 ('굿즈 더만들어주세요 사진의', 0.5824)]

롱스티커

In [ ]:
#긍정 전처리
df1_롱스티커["cut_special_character"]=df1_롱스티커['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df1_롱스티커["cut_korean_consonant"]=df1_롱스티커["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df1_롱스티커["fiterd_data"]=df1_롱스티커["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

#부정 전처리
df0_롱스티커["cut_special_character"]=df0_롱스티커['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df0_롱스티커["cut_korean_consonant"]=df0_롱스티커["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df0_롱스티커["fiterd_data"]=df0_롱스티커["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

<ipython-input-35-e112f9b1ff03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_롱스티커["cut_special_character"]=df1_롱스티커['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
<ipython-input-35-e112f9b1ff03>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_롱스티커["cut_korean_consonant"]=df1_롱스티커["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
<ipython-input-35-e112f9b1ff03>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
review1 = listToString(df1_롱스티커['fiterd_data'])
review0 = listToString(df0_롱스티커['fiterd_data'])

In [ ]:
!pip install keybert -q

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

doc1 = review1 #긍정
doc0 = review0 #부정

In [ ]:
print(len(doc1), len(doc0))

25143 5329


In [ ]:
#MMR(Maximal Margimal Relevance) 긍정
keywords_mmr_1 = kw_model.extract_keywords(doc1,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_1

[('보관중입니다 마루너무너무너무너무 귀여워요 동생', 0.5683),
 ('귀여워요 보관중입니다 마루너무너무너무너무', 0.5621),
 ('완전 대만족입니다 마루는 사랑이어서', 0.561),
 ('귀여워요 보관중입니다 마루너무너무너무너무 귀여워요', 0.5605),
 ('대만족입니다 마루는 사랑이어서 조아요', 0.5377),
 ('말랑큐티깜찍 마루스티커 완전 대만족입니다', 0.5324),
 ('진짜 너무 귀여워요 보관중입니다', 0.5259),
 ('대만족 마루는 귀엽다 다이어리에도', 0.521),
 ('귀여워서 와글와글 마루가 너무', 0.5156),
 ('나와여 우주최강기요미마루 행복해요 마루', 0.5116),
 ('데굴데굴 굴러가는걸 만들 생각을', 0.5097),
 ('귀여워요 사진과 같이 보이는대로', 0.5066),
 ('붙엿는데 너뮤기여어 넘넘기여워요 마루가', 0.5063),
 ('지인분들과 공동구매했는데 모두 만족하고', 0.5047),
 ('마루야 너의 행복이 행복이다', 0.5036),
 ('저는 만족해요 너무', 0.4986),
 ('흔들어주던 고구마루 날아가는 고구마루', 0.49),
 ('2개는 보관용 1개는 쓰려고', 0.4827),
 ('다양한 표정의 마루 모두', 0.4758),
 ('완전 귀엽고 다꾸하기에 완전', 0.4749)]

In [ ]:
#MMR(Maximal Margimal Relevance) 부정
keywords_mmr_0 = kw_model.extract_keywords(doc0,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_0

[('내주세요 여자친구가나무조아해요덜덜 너무', 0.579),
 ('만들어주세요 생각보다는 작지만 너무', 0.5759),
 ('데굴데굴 굴러가는걸 만들 생각을', 0.5736),
 ('비싸지만 작가님게 한톨의 도움이라도', 0.5707),
 ('키링도 만들어주세요 생각보다는 작지만', 0.5683),
 ('스릉흔드 생각보다 너무너무 작아요', 0.5593),
 ('비해 비싸지만 작가님게 한톨의', 0.55),
 ('작기는 한데 소장용이라서 만족합니다', 0.5492),
 ('생각보다 작아용 제발 두개씩', 0.5442),
 ('사이즈가 작기는 한데 소장용이라서', 0.5429),
 ('예뻐요 스릉흔드 생각보다 너무너무', 0.5372),
 ('조아요 많이 만들어주세여 생각보단', 0.5368),
 ('너무너무 귀여워요 상세설명 안봐서', 0.5331),
 ('내주시와요 표정마루는 품절이라 아쉽지만', 0.5308),
 ('만들어주세요 마루 이모티콘도 만들어주세요', 0.5294),
 ('롱스티커는 사이즈를 대충 봐서', 0.5286),
 ('만들어주세요 생각보다 작긴 한데', 0.5216),
 ('사무용품 마루로 만들어주세요 마루', 0.5192),
 ('얼른 다른굿즈들 가져와주세여 저의', 0.5169),
 ('강쥐마루 보는데 마루 특유의', 0.4992)]

미니 인형키링

In [ ]:
#긍정 전처리
df1_인형키링["cut_special_character"]=df1_인형키링['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df1_인형키링["cut_korean_consonant"]=df1_인형키링["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df1_인형키링["fiterd_data"]=df1_인형키링["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

#부정 전처리
df0_인형키링["cut_special_character"]=df0_인형키링['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df0_인형키링["cut_korean_consonant"]=df0_인형키링["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df0_인형키링["fiterd_data"]=df0_인형키링["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

<ipython-input-28-66f106f67638>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_인형키링["cut_special_character"]=df1_인형키링['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
<ipython-input-28-66f106f67638>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_인형키링["cut_korean_consonant"]=df1_인형키링["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
<ipython-input-28-66f106f67638>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
review1 = listToString(df1_인형키링['fiterd_data'])
review0 = listToString(df0_인형키링['fiterd_data'])

In [ ]:
!pip install keybert -q

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

doc1 = review1 #긍정
doc0 = review0 #부정

In [ ]:
print(len(doc1), len(doc0))

14799 13047


In [ ]:
#MMR(Maximal Margimal Relevance) 긍정
keywords_mmr_1 = kw_model.extract_keywords(doc1,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_1

[('사랑스러운 마루입니다 손바닥 정도의', 0.586),
 ('아름답고 그렇습니다 마음의 안정과', 0.5703),
 ('귀엽네요 만화 장면이 생각나서', 0.5602),
 ('너어어어무 귀여어요 푸바오와 마루와', 0.5513),
 ('마음에 들어요 마루가 당장이라도', 0.5512),
 ('행복을 가져다주는 마루 미용실', 0.5496),
 ('안정과 평화를 가져다주는 마루네요', 0.5492),
 ('마음의 안정과 평화를 가져다주는', 0.5478),
 ('생일선물로 줬는데 매우 만족스럽습니다', 0.5467),
 ('마루 착용감 대박입니다 미용실', 0.5465),
 ('후회중 마루인형 완전', 0.5432),
 ('인형은 너무너무 만족스러워요 너무너무', 0.5423),
 ('작고요 손에 들어와요 마루', 0.5365),
 ('맘에들어요 진짜너무귀엽고무엇보다 보면 행복해진다는점이', 0.5358),
 ('완전 귀엽고 부들부들합니다 온라인', 0.5301),
 ('귀엽죠 언제어디서나 마루와', 0.529),
 ('마루는 행복을 주는 강아지구나', 0.5236),
 ('아담한 사이즈 마루최고 기엽고요기엽고참기엽습니다', 0.521),
 ('너무너무너무너무너무 귀여워요 앨리스 마루라니', 0.5069),
 ('행복해져요 귀여움 한도초과', 0.5043)]

In [ ]:
#MMR(Maximal Margimal Relevance) 부정
keywords_mmr_0 = kw_model.extract_keywords(doc0,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_0

[('하는 혼자만의 스토리텔링으로 매관매직', 0.6238),
 ('저희 마루는 인생이', 0.6116),
 ('만들어주새오 나머지는 품절이라 너무', 0.6073),
 ('적당하고 완전 만족스러워요 너무너무너무너무너무너무', 0.602),
 ('마감이 별로더라구요 마루 흰자에', 0.5992),
 ('사전공지가 매우매우매우 중요합니다 그리고', 0.5979),
 ('마루 얼굴보자마자 완전 행복해지면서', 0.5951),
 ('굿즈 만들어주세용 아무래도 공장에서', 0.593),
 ('이염이 있었는데 문의해도 느리고', 0.5856),
 ('둘러서 배송하는데 무성의', 0.583),
 ('생각보다 만족스럽네요 마루라서', 0.5818),
 ('턱주가리에요 마감은 하지만 마루는', 0.5814),
 ('탈부착이에요 묭실마루 미쳣다 늑대마루', 0.581),
 ('무시하자라고 생각하시고 계신다면 작가님과', 0.5806),
 ('마감이 아쉽네요 판매와 배송에서', 0.5802),
 ('품절이라 누군가의 취소분을 극적으로', 0.5783),
 ('생각하시고 계신다면 작가님과 팬분들에게', 0.5781),
 ('늑대마루도 갖고싶었는데 품절이라 누군가의', 0.5728),
 ('인형부분은 생각보다 만족스럽네요 마루라서', 0.562),
 ('교환은 했지만 다리 비대칭', 0.5511)]

스프링노트

In [ ]:
#긍정 전처리
df1_스프링노트["cut_special_character"]=df1_스프링노트['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df1_스프링노트["cut_korean_consonant"]=df1_스프링노트["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df1_스프링노트["fiterd_data"]=df1_스프링노트["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

#부정 전처리
df0_스프링노트["cut_special_character"]=df0_스프링노트['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df0_스프링노트["cut_korean_consonant"]=df0_스프링노트["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df0_스프링노트["fiterd_data"]=df0_스프링노트["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

<ipython-input-19-3124ddbcd8bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_스프링노트["cut_special_character"]=df1_스프링노트['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
<ipython-input-19-3124ddbcd8bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_스프링노트["cut_korean_consonant"]=df1_스프링노트["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
<ipython-input-19-3124ddbcd8bd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
review1 = listToString(df1_스프링노트['fiterd_data'])
review0 = listToString(df0_스프링노트['fiterd_data'])

In [ ]:
!pip install keybert -q

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

doc1 = review1 #긍정
doc0 = review0 #부정

In [ ]:
print(len(doc1), len(doc0))

8609 1561


In [ ]:
#MMR(Maximal Margimal Relevance) 긍정
keywords_mmr_1 = kw_model.extract_keywords(doc1,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_1

[('참고하세요 초등고학년 아이와 고등학생', 0.6074),
 ('색상 참고하세요 초등고학년 아이와', 0.6019),
 ('직접 마루와 친구들을 그려주었습니당', 0.5749),
 ('직장인인데 마루 행복해하는 모습에', 0.5635),
 ('공부안하는 직장인인데 마루 행복해하는', 0.5622),
 ('마루 한방울 한방울에 만족의', 0.5492),
 ('행복해요 무선 유선', 0.5441),
 ('것처럼 실생활에 유용합니다 마루가', 0.5435),
 ('보이네요 올라와 있는 색상', 0.5416),
 ('억억 마루해피도 억억', 0.5412),
 ('보이는 것처럼 실생활에 유용합니다', 0.5405),
 ('이잉 마루공책으로 열공할게요 엄청', 0.537),
 ('대체로 만족합니다 친구 선물주려고', 0.5356),
 ('빠른 출고해주신 덕에 아침', 0.5299),
 ('선물했는데 정말 마음에 들어해서', 0.5256),
 ('서율이 실전화기', 0.5233),
 ('전해달래요 좋습니다 대체로 만족합니다', 0.5216),
 ('와서 너무 만족해요 너무너무', 0.5169),
 ('강쥐빼꼼은 무선이라 완전 원하던', 0.5157),
 ('덕에 아침 8시에 도착한거', 0.4846)]

In [ ]:
#MMR(Maximal Margimal Relevance) 부정
keywords_mmr_0 = kw_model.extract_keywords(doc0,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_0

[('쓸만해요 마루덕후 친구 생일선물로', 0.5622),
 ('안살수가 없었어요 표지는 무광이고', 0.5372),
 ('쓸만해요 마루덕후 친구', 0.5347),
 ('약간흐물한데 쓸만해요 마루덕후 친구', 0.5341),
 ('마루의 격함을 좋아하는 친군데', 0.5338),
 ('없었어요 표지는 무광이고', 0.5298),
 ('친구 생일선물로 잔뜩드려보앗습니다 마루의', 0.5232),
 ('생일선물로 잔뜩드려보앗습니다 마루의 격함을', 0.5189),
 ('표지는 무광이고 스크래치 나는', 0.5179),
 ('귀여우니 조금 싸구려 질의', 0.5144),
 ('만들어주세요 작가님 에바입니다 마루', 0.5124),
 ('만들어주세요 마루 이모티콘도 만들어주세요', 0.5123),
 ('약간흐물한데 쓸만해요 마루덕후', 0.5097),
 ('생일선물로 잔뜩드려보앗습니다 마루의', 0.507),
 ('얼른 다른굿즈들 가져와주세여 저의', 0.5014),
 ('세안밴드 출시해주세요 살게요 너무너무', 0.4956),
 ('흐물거림 약간흐물한데 쓸만해요 마루덕후', 0.4937),
 ('이모티콘도 만들어주세요 작가님 에바입니다', 0.4849),
 ('갈색이라 별로임 마루 레전드', 0.4707),
 ('저흰 서른이에요 마루야 이모들이', 0.4691)]

사각필통

In [ ]:
#긍정 전처리
df1_사각필통["cut_special_character"]=df1_사각필통['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df1_사각필통["cut_korean_consonant"]=df1_사각필통["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df1_사각필통["fiterd_data"]=df1_사각필통["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

#부정 전처리
df0_사각필통["cut_special_character"]=df0_사각필통['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
df0_사각필통["cut_korean_consonant"]=df0_사각필통["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
df0_사각필통["fiterd_data"]=df0_사각필통["cut_korean_consonant"].apply(lambda x: re.sub(r"[a-z]([A-Z])", r"-\1", x).upper())

<ipython-input-7-77780479f84a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_사각필통["cut_special_character"]=df1_사각필통['review'].apply(lambda x: re.sub(r"[^ ㄱ-ㅣ가-힣A-Za-z0-9]", " ", x))
<ipython-input-7-77780479f84a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_사각필통["cut_korean_consonant"]=df1_사각필통["cut_special_character"].apply(lambda x: re.sub(r"([ㄱ-ㅎㅏ-ㅣ]+)", " ", x))
<ipython-input-7-77780479f84a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
review1 = listToString(df1_사각필통['fiterd_data'])
review0 = listToString(df0_사각필통['fiterd_data'])

In [ ]:
!pip install keybert -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

doc1 = review1 #긍정
doc0 = review0 #부정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(len(doc1), len(doc0))

6306 832


In [ ]:
#MMR(Maximal Margimal Relevance) 긍정
keywords_mmr_1 = kw_model.extract_keywords(doc1,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_1

[('성인이지만 마루를 애정하는 마음으로', 0.5722),
 ('새학기를 책임져주라 친언니 생일선물로', 0.5676),
 ('엄청엄청 들어갑니다 조만간 갈색마루도', 0.5541),
 ('마루를 애정하는 마음으로 열심히', 0.553),
 ('조아하네요 마루덕후 친구 생일선물로', 0.5409),
 ('전완근사이즈여서 학용품으로는', 0.5335),
 ('갈색 색깔도 어울려요 모의고사의', 0.5183),
 ('공간이 부족해서 화이트나 메모지를', 0.514),
 ('색도 기대한 색감이에요 너무', 0.5102),
 ('친구 생일선물로 잔뜩드려보앗습니다 마루의', 0.5064),
 ('만족스럽습니다 마루 귀여워요 사각필통', 0.5062),
 ('재질도 좋아요 아주 만족스럽습니다', 0.506),
 ('모의고사의 맛을 잊을 정도로', 0.5058),
 ('포인트예요 아들이 친구 생일선물한다고', 0.503),
 ('귀여워요 마루의 콩알만한 눈이', 0.495),
 ('마루 새학기를 책임져주라 친언니', 0.4944),
 ('좋아요 너무너무너무너무너무너무너묵 귀야으 강추입니다', 0.4917),
 ('들어가서 좋습니다 지퍼에도 마루가', 0.4911),
 ('들어갑니다 조만간 갈색마루도', 0.4903),
 ('모의고사의 맛을 잊을', 0.4757)]

In [ ]:
#MMR(Maximal Margimal Relevance) 부정
keywords_mmr_0 = kw_model.extract_keywords(doc0,keyphrase_ngram_range=(2,4),use_mmr = True,top_n = 20,diversity = 0.3)
keywords_mmr_0

[('만들어주세여 맘에 드는 디자인', 0.5318),
 ('마루로 만들어주세요 배송 너무느림', 0.5291),
 ('마루로 만들어주세요 배송', 0.5055),
 ('너무느림 해외배송인듯합니다 아주아주 별로입니다', 0.5025),
 ('만들어주세요 배송', 0.5007),
 ('사무용품 마루로 만들어주세요 배송', 0.4994),
 ('만들어주세요 배송 너무느림 해외배송인듯합니다', 0.4968),
 ('아주아주 별로입니다 갈색이라 더러워질', 0.4911),
 ('빵빵해요 일반 고등학생이 들고다닐', 0.4897),
 ('아주아주 별로입니다 갈색이라', 0.4887),
 ('많이 만들어주세여 맘에 드는', 0.4864),
 ('들어갈것같진 않고 무난해요 직접', 0.4808),
 ('들어가요 마루가 그냥 엄청', 0.4763),
 ('되고 사이즈도 적당하고 너무너무', 0.4753),
 ('사용 중입니다 귀여워요 마루', 0.4735),
 ('조아요 많이 만들어주세여 맘에', 0.4703),
 ('얼굴이라 좋아요 화면이랑 진짜', 0.4624),
 ('마루가 너무 귀여워요 노란색은', 0.4579),
 ('해도 되고 사이즈도 적당하고', 0.4466),
 ('좋아요 저는 학생이', 0.4322)]